# AutoGen Einfaches Beispiel

In diesem Codebeispiel verwenden Sie das [AutoGen](https://aka.ms/ai-agents/autogen) KI-Framework, um einen einfachen Agenten zu erstellen.

Das Ziel dieses Beispiels ist es, Ihnen die Schritte zu zeigen, die wir später in den zusätzlichen Codebeispielen verwenden werden, wenn wir die verschiedenen agentischen Muster implementieren.


## Importieren Sie die benötigten Python-Pakete


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Erstellen des Clients

In diesem Beispiel verwenden wir [GitHub Models](https://aka.ms/ai-agents-beginners/github-models), um auf das LLM zuzugreifen.

Das `model` ist als `gpt-4o-mini` definiert. Versuchen Sie, das Modell auf ein anderes im GitHub Models-Marktplatz verfügbares Modell zu ändern, um unterschiedliche Ergebnisse zu sehen.

Als schnellen Test führen wir einfach eine einfache Eingabeaufforderung aus – `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definition des Agenten

Nachdem wir den `client` eingerichtet und bestätigt haben, dass er funktioniert, erstellen wir einen `AssistantAgent`. Jedem Agenten können folgende Eigenschaften zugewiesen werden:  
**name** - Ein kurzer Name, der nützlich ist, um ihn in Multi-Agent-Workflows zu referenzieren.  
**model_client** - Der Client, den Sie im vorherigen Schritt erstellt haben.  
**tools** - Verfügbare Werkzeuge, die der Agent nutzen kann, um eine Aufgabe zu erledigen.  
**system_message** - Die Metaprompt, die die Aufgabe, das Verhalten und den Ton des LLM definiert.  

Sie können die Systemnachricht ändern, um zu sehen, wie das LLM reagiert. Wir werden `tools` in Lektion #4 behandeln.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Agent ausführen

Die folgende Funktion wird den Agenten ausführen. Wir verwenden die Methode `on_message`, um den Zustand des Agenten mit der neuen Nachricht zu aktualisieren.

In diesem Fall aktualisieren wir den Zustand mit einer neuen Nachricht vom Benutzer, die lautet: `"Plane mir einen großartigen sonnigen Urlaub"`.

Du kannst den Nachrichteninhalt ändern, um zu sehen, wie das LLM unterschiedlich reagiert.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
